In [18]:
import pandas as pd
from openai import OpenAI
# import json

In [4]:
df = pd.read_csv('D:\\Metaverse_Museum\\AI-Curators\\Recommend\\output\\User001\\2024-08-05_17-26-27.359981\\Exhibition 1.csv')

In [109]:
df2 = pd.read_csv('D:\\Metaverse_Museum\\AI-Curators\\Recommend\\output\\User001\\2024-08-05_17-26-27.359981\\Exhibition 2.csv')

In [100]:
# Save the titles and artists of the dataframe into a string in the format of "title1, artist1; title2, artist2; ..."
exhibition = ""
for index, row in df.iterrows():
    exhibition += row['title'] + ", " + row['artist_display'] + "; "

In [110]:
# Save the titles and artists of the dataframe into a string in the format of "title1, artist1; title2, artist2; ..."
exhibition2 = ""
for index, row in df2.iterrows():
    exhibition2 += row['title'] + ", " + row['artist_display'] + "; "

In [114]:
exhibition2

"Woman with a Parasol - Madame Monet and Her Son, Claude Monet (French, 1840-1926); Haystacks, Georges Seurat (French, 1859-1891); Woman Bathing Her Feet in a Brook, Camille Pissarro (French, 1830-1903); The Olive Orchard, Vincent van Gogh (Dutch, 1853-1890); Scenes from a Legend, Giovanni Larciani (Master of the Kress Landscapes) (Florentine, 1484-1527); The Poet's Garden, Vincent van Gogh (Dutch, 1853-1890); Stack of Wheat (Thaw, Sunset), Claude Monet (French, 1840-1926); Man with a Hoe, Georges Seurat (French, 1859-1891); The Artist's Garden at V��theuil, Claude Monet (French, 1840-1926); Meadow, Alfred Sisley (French, 1839-1899); Landscape with Figures, Maurice Prendergast (American, 1858-1924); The Artist's House at Argenteuil, Claude Monet (French, 1840-1926); The Willows, Claude Monet (French, 1840-1926); "

In [113]:
key = 'key' # your openai key
client = OpenAI(api_key=key)

In [115]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are a professional art exhibition curator.  You can give an accurate, straightforward, and informative description for an exhibition of artworks for the general public to understand. An exhibition can be themed based on artists, genre, style, period, color, or any other factors that are shared by the artworks in the exhibition.\nYou will be given a list of artwork names and the corresponding artists. Provide a Python readable JSON string to describe the artwork with the following keys and values:\ntitle: <string, in 15 words> An elegant name for the exhibition of artworks,\ndescription: <string, in 200 words> a paragraph that introduces the themed exhibition to viewers,\nartworks_orders: <list of strings, ordered> rearrange the order of the input list of artworks for viewers to understand the exhibition."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{exhibition}"
        }
      ]
    }  
  ],
  temperature=0,
  max_tokens=512,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [123]:
response

ChatCompletion(id='chatcmpl-9t2M27xS95g40f8kbsQ8r12UDckxr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "title": "Impressionist Visions: Nature and Still Life in Art",\n  "description": "This exhibition showcases a curated selection of Impressionist artworks that celebrate the beauty of nature and the simplicity of still life. Featuring renowned artists such as Claude Monet, Vincent van Gogh, and Pierre-Auguste Renoir, the collection highlights the movement\'s characteristic use of light, color, and brushwork. Visitors will experience a diverse range of subjects, from serene landscapes and vibrant floral arrangements to intimate still life compositions. Each piece reflects the artists\' innovative approaches to capturing fleeting moments in time, inviting viewers to appreciate the interplay between nature and everyday life. The exhibition aims to immerse the audience in the Impressionist ethos, emphasizing the impor

In [105]:
result = response.choices[0].message.content.split("\n")

In [106]:
exhibition_temp = {}
get_artworks = False
artworks = []
for line in result:
    if not get_artworks:
        if "title" in line:
            exhibition_temp['title'] = line.split('": ')[1].replace('"', '').replace(", ", "")
        elif "description" in line:
            exhibition_temp['description'] = line.split('": ')[1].replace('"', '').replace(", ", "")
        elif "artworks_orders" in line:
            get_artworks = True
    else:
        if ']' in line:
            exhibition_temp['artworks_orders'] = artworks
            break
        else:
            title = line.split(", ")[0]
            artworks.append(title.replace('"', '').replace(",", "").lstrip())

In [108]:
exhibition_temp

{'title': 'The Essence of Still Life: A Journey Through Time and Color,',
 'description': "This exhibition'The Essence of Still Life,' invites viewers to explore the rich tradition of still life paintingshowcasing the works of renowned artists from the 19th and early 20th centuries. Each piece reflects the artists' unique interpretations of everyday objectstransforming them into captivating visual narratives. From Cezanne's masterful compositions to Monet's vibrant floral arrangementsthe exhibition highlights the interplay of lightcolorand form. Visitors will encounter a diverse array of subjectsincluding fruitsflowersand domestic sceneseach meticulously crafted to evoke emotion and provoke thought. The exhibition also features works by Renoir and Manetwho brought their own distinctive styles to the genre. By examining these artworksviewers will gain insight into the cultural and historical contexts that shaped the still life traditionas well as the innovative techniques employed by th

In [52]:
# Clean the JSON string
def clean_json_string(json_str):
    # Replace any unwanted escape characters if necessary
    json_str = json_str.replace(r'\n', '')  # Remove newline escape sequences if they are causing issues
    json_str = json_str.replace(r'\t', '')  # Remove tab escape sequences if they are causing issues
    return json_str

In [53]:
# Clean and parse the JSON string
cleaned_json_string = clean_json_string(result)

In [54]:
cleaned_json_string

'```json\n{\n  "title": "The Essence of Still Life: A Journey Through Time and Color",\n  "description": "This exhibition, \'The Essence of Still Life,\' invites viewers to explore the rich tradition of still life painting, showcasing works from renowned artists such as Paul Cézanne, Claude Monet, and Pierre-Auguste Renoir. Each piece reflects the artists\' unique interpretations of everyday objects, transforming them into captivating visual narratives. From Cézanne\'s iconic \'Three Pears\' to Monet\'s vibrant \'Jerusalem Artichoke Flowers,\' the exhibition highlights the evolution of still life across different periods and styles. The careful arrangement of fruits, flowers, and everyday items not only demonstrates technical skill but also evokes deeper themes of beauty, transience, and the passage of time. Visitors will appreciate the interplay of color, light, and form as they journey through the works of these masterful artists. The exhibition aims to foster a greater understanding

In [55]:
# data = json.loads(cleaned_json_string)

In [56]:
# convert the response string into a dictionary


In [58]:
# save response to a json file and remove the ``` from the string
with open('D:\\Metaverse_Museum\\AI-Curators\\Recommend\\output\\User001\\2024-08-05_17-26-27.359981\\exhibition_description.json', 'w') as f:
    f.write(json.dumps(response.choices[0].message.content))
f.close()    

In [59]:
# write the response to a txt file
with open('D:\\Metaverse_Museum\\AI-Curators\\Recommend\\output\\User001\\2024-08-05_17-26-27.359981\\exhibition_description.txt', 'w') as f:
    f.write(result)
f.close()

In [66]:
# read the txt file line by line and save it to a dictionary
with open('D:\\Metaverse_Museum\\AI-Curators\\Recommend\\output\\User001\\2024-08-05_17-26-27.359981\\exhibition_description.txt', 'r') as f:
    lines = f.readlines()
    exhibition_dict = {}
    for line in lines:
        if ':' not in line:
            continue
        key, value = line.split(':')
        exhibition_dict[key] = value
f.close()

ValueError: too many values to unpack (expected 2)

['```json\n',
 '{\n',
 '  "title": "The Essence of Still Life: A Journey Through Time and Color",\n',
 '  "description": "This exhibition, \'The Essence of Still Life,\' invites viewers to explore the rich tradition of still life painting, showcasing works from renowned artists such as Paul Cézanne, Claude Monet, and Pierre-Auguste Renoir. Each piece reflects the artists\' unique interpretations of everyday objects, transforming them into captivating visual narratives. From Cézanne\'s iconic \'Three Pears\' to Monet\'s vibrant \'Jerusalem Artichoke Flowers,\' the exhibition highlights the evolution of still life across different periods and styles. The careful arrangement of fruits, flowers, and everyday items not only demonstrates technical skill but also evokes deeper themes of beauty, transience, and the passage of time. Visitors will appreciate the interplay of color, light, and form as they journey through the works of these masterful artists. The exhibition aims to foster a great

In [ ]:
# function that transform a string of JSON into a dictionary
def json_to_dict(json_str):
    json_str = json_str.replace('title', '"title"').replace('description', '"description"').replace('artworks_orders', '"artworks_orders"')
    json_str = json_str.replace(';', ',').replace('Three Pears, Paul C��zanne (French, 1839-1906)', 'Three Pears, Paul Cézanne (French, 1839-1906)')
    